# Tube Turbulent with 2 groups IATE with thermal effects - Validation

# Description

A two-group wall-boiling model has been implemented and coupled with the two-group IATE in Validation of two-group interfacial area transport equation in boiling flow, J.L. Bottini et al. (2024, https://doi.org/10.1016/j.ijheatmasstransfer.2024.125515.). This integrated model is compared against experimental data because the previous paper performed only a 1D computation. The 1D comparison showed that the greatest sensitivity stems from uncertainties in the underlying model, rather than experimental uncertainty in the boundary conditions. The model's performance is particularly challenged under conditions of high heat flux and subcooling, where inaccuracies in predicting the group-1 Sauter mean diameter lead to delayed group-2 vapor formation and an overprediction in group-1 interfacial area concentration.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
matplotlib.rcParams['text.usetex'] = True
import copy
import os, sys
from string import Template
import time
from IPython.display import display
import math

from trustutils import run 
from trustutils import visit
from trustutils.jupyter import plot
plt.rcParams["font.weight"] = "bold"

run.TRUST_parameters()

# Definition of probes

In [ ]:
sondes = { }

n_seg_sonde = 50
r_tube = 0.009525

tab_name_sondes = ["vitesse_liquide_water",
                   "vitesse_gaz_water_group1",
                   "vitesse_gaz_water_group2",
                   "alpha_gaz_water_group1",
                   "alpha_gaz_water_group2",
                   "pression",
                   "dp",
                   "d_bulles",
                   "interfacial_area",
                   "temperature_liquide_water",
                   "temperature_gaz_water_group1",
                   "temperature_gaz_water_group2"
                  ]

x_0 = 0.0
x_end = r_tube
y_0 = 0.
y_end = 0.
z_0 = 0.2

z_end = 3.2


str_sonde = " "
for name_sonde in tab_name_sondes:
    str_sonde += f" {name_sonde} {name_sonde} periode 1.e-2 segment {n_seg_sonde} {x_0} {y_0} {z_0} {x_0} {y_0} {z_end}  \n"

zexp=[0+z_0,1.114+z_0,1.815+z_0,2.217+z_0,2.60+z_0,2.98+z_0]
for i in zexp : 
    str_sonde += f" alphag1_{i} alpha_gaz_water_group1 periode 1.e-2 segment {n_seg_sonde} {x_0} {y_0} {i} {x_end} {y_end} {i}  \n"
    str_sonde += f" alphag2_{i} alpha_gaz_water_group2 periode 1.e-2 segment {n_seg_sonde} {x_0} {y_0} {i} {x_end} {y_end} {i}  \n"
    str_sonde += f" ai_{i} interfacial_area periode 1.e-2 segment {n_seg_sonde} {x_0} {y_0} {i} {x_end} {y_end} {i}  \n"

sondes["PolyMAC"] = str_sonde

# Preparation and launching of the computation

In [ ]:
force_recalculation = True 
number_of_partitions = 4

# Mesh
med_file   = {   "PolyMAC" : "1_canal_1.med" }
name_mesh   = {"PolyMAC" : "read_med { domain dom mesh mesh file 1_canal_1.med }", "VDF" : "Mailler dom { Pave tube  { Origine 0. 0. Nombre_de_Noeuds 3 336 Longueurs  0.01905 3.4 } { bord symetrie    X = 0   0 <= Y <= 3.4  bord top   Y = 3.4   0 <= X <= 0.01905  bord bottom    Y = 0   0 <= X <= 0.01905 bord wall_cold1   X = 0.01905 0 <= Y <= 0.2 bord wall_hot   X = 0.01905 0.2 <= Y <= 3.2 bord wall_cold2   X = 0.01905 3.2 <= Y <= 3.4  } } " }

schema = { "PolyMAC" : " PolyMAC_P0  dis option_PolyMAC { traitement_axi } " }
dimension = { "PolyMAC" : 3}
bidim_axi = {  "PolyMAC" : " "}
nrad = {  "PolyMAC" : 40}
solveur = {  "ICE" : "" }
paroi = {   "PolyMAC" : " paroi "}
paroi_adiabatique = {   "PolyMAC" : " paroi_adiabatique "}
z_grav = {  "PolyMAC" : "z"}

# Numerical parameters
facsec  = 0.5
tmax    = 10
nb_pas_dt_max = '20000'
seuil_statio  = '1.e-3'

#Physical quantities
u_0 = 1.277

#Turbulent quantities
D_h = 0.01905
CI_om = u_0/D_h
CI_k  = 0.03*u_0**2
        
#Two-phase quantities
frottement_interfacial = " Kumar_Brooks { Dh 0.01905 } " 
db1 = 0.0001
db2 = 0.011
Cw1 = -.1
Cw2 = .147
ai1 = 6.*0.00001/db1
ai2 = 6.*0.00001/db2
Tg=138.3
dT=5.23
Tl=130
qw=158400
P=348700-820*9.81*3.4

build_dir = run.BUILD_DIRECTORY

if force_recalculation or not os.path.exists(f'{build_dir}'):
    print('Recalculation running..')
    run.reset()

    for s in schema.keys() :
        for solv in solveur.keys() :
            name = f"{s}_{solv}"

            substitutions_dict = {  "dimension" : dimension[s],
                                    "bidim_axi" : bidim_axi[s],
                                    "name_mesh" : f'{name_mesh[s]}' ,
                                    "schema" : schema[s],
                                    "z": z_grav[s],
                                    "tmax": str(tmax),                                  
                                    "facsec" : str(facsec) ,
                                    "nb_pas_dt_max" : nb_pas_dt_max ,
                                    "seuil_statio" : seuil_statio ,
                                    "solveur" : solv ,
                                    "facsec_diffusion_for_sets" : solveur[solv] ,
                                  
                                    "paroi_adiabatique" : paroi_adiabatique[s],
                                    "paroi" : paroi[s],
                                                                                               
                                    "diametre_bulles" : str(db1),
                                    "u_0"  : f"{3*dimension[s]} {''.join([ ' 0 ' for i in range(3*(dimension[s]-1))])} {u_0} {u_0} {u_0} " ,
                                    "grav"   : f"{3*dimension[s]} {''.join([ ' 0 ' for i in range(3*(dimension[s]-1))])} -9.81 -9.81 -9.81 " ,
                                    "frottement_interfacial" : frottement_interfacial,
                                                
                                    "CI_diss": f"{CI_om}" ,
                                    "CI_k": f"{CI_k}" ,

                                    "ci_ai1" : str(ai1),
                                    "ci_ai2" : str(ai2),
                                    "Tl0" : str(Tl),
                                    "Tg0" : str(Tg),
                                    "dT0" : str(dT),
                                    "Tg1" : str(Tg-dT),
                                    "Tl1" : str(Tl),
                                    "P_surf" : str(qw),
                                    "Press" : str(P),

                                    "sondes" : sondes[s],
                                                
                                }

            tc = run.addCaseFromTemplate("jdd.data",
                                         targetDirectory=f"{name}",
                                         dic=substitutions_dict,
                                         nbProcs=number_of_partitions)

            if (med_file[s]!=""):
                os.system(f'cp {build_dir}/{med_file[s]} {build_dir}/{name}/{med_file[s]}')
            if number_of_partitions > 1:
                tc.partition()

    run.printCases()
    run.runCases()
    display(run.tablePerf())

else:
    print('No recalculation, reusing old results!')

# Loading results

In [ ]:
############# results : sim[config][debit air][raf][position]
#############         columns : ['y/h'] ['vxl'] ['vyl'] ['vxg'] ['vyg'] ['k'] ['alp'] ['DB']
#############                   ["diff"] ["conv"] ["press"] ["grav"] ["drag"] ["lift"] ["disp"] ["lub"]
#############               nb : end column name with lx for forces
############# sim name : tab_name [config][debit air][raf]
############# sim label: tab_label[config][debit air][raf]
############# convergence :  dt_ev[config][debit air][raf]
#############         columns : ['time']['dt']['facsec']['residu']['dt_stab']['vitl']['vitg']['alphal']['alphag']['Tl']['Tg']['diss']['k'] 

sim      = {  }
dt_ev    = {  }
par = ""
if number_of_partitions > 1 : par = "PAR_"

for s in schema.keys() :
    for solv in solveur.keys() :
        name = f"{s}_{solv}"

        dt_ev[name] = pd.read_csv(f"{build_dir}/{name}/{par}jdd.dt_ev", sep="\t")
        dt_ev[name] = dt_ev[name].iloc[:,:14]
        dt_ev[name].columns = ['time', 'dt', 'facsec', 'residu', 'dt_stab', 'vitl', 'vitg1','vitg2', 'alphal', 'alphag1', 'alphag2', 'Tl', 'Tg1','Tg2']

        sim[name] = pd.DataFrame()
        sim[name]['z']  = np.linspace(z_0, z_end, n_seg_sonde)
        sim[name]['vxl'] = np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_WATER.son"))[1::dimension[s],-1]
        sim[name]['vzl'] = np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_LIQUIDE_WATER.son"))[dimension[s]::dimension[s],-1]
        sim[name]['vxg1'] = np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_WATER_GROUP1.son"))[1::dimension[s],-1]
        sim[name]['vzg1'] = np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_WATER_GROUP1.son"))[dimension[s]::dimension[s],-1]
        sim[name]['vxg2'] = np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_WATER_GROUP2.son"))[1::dimension[s],-1]
        sim[name]['vzg2'] = np.array(plot.loadText(f"{name}/{par}jdd_VITESSE_GAZ_WATER_GROUP2.son"))[dimension[s]::dimension[s],-1]
        sim[name]['alp1'] = np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_WATER_GROUP1.son"))[1::,-1]
        sim[name]['alp2'] = np.array(plot.loadText(f"{name}/{par}jdd_ALPHA_GAZ_WATER_GROUP2.son"))[1::,-1]
        sim[name]['p']   = np.array(plot.loadText(f"{name}/{par}jdd_PRESSION.son"))[1::,-1]
        sim[name]['dp']  = np.array(plot.loadText(f"{name}/{par}jdd_DP.son"))[1::,-1]
        sim[name]['db1']  = np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES.son"))[1::,-1][1::3]
        sim[name]['db2']  = np.array(plot.loadText(f"{name}/{par}jdd_D_BULLES.son"))[1::,-1][2::3]
        sim[name]['ai1']  = np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA.son"))[1::,-1][1::3]
        sim[name]['ai2']  = np.array(plot.loadText(f"{name}/{par}jdd_INTERFACIAL_AREA.son"))[1::,-1][2::3]
        sim[name]['Tl'] = np.array(plot.loadText(f"{name}/{par}jdd_TEMPERATURE_LIQUIDE_WATER.son"))[1::,-1]
        sim[name]['Tg1'] = np.array(plot.loadText(f"{name}/{par}jdd_TEMPERATURE_GAZ_WATER_GROUP1.son"))[1::,-1]
        sim[name]['Tg2'] = np.array(plot.loadText(f"{name}/{par}jdd_TEMPERATURE_GAZ_WATER_GROUP1.son"))[1::,-1]
        
        if (solv == "ICE") :
            al1=[]
            al2=[]
            ai1=[]
            ai2=[]
            for i in zexp : 
                al1.append(np.mean(np.array(plot.loadText(f"{name}/{par}jdd_ALPHAG1_{i}.son"))[1::,-1]))
                al2.append(np.mean(np.array(plot.loadText(f"{name}/{par}jdd_ALPHAG2_{i}.son"))[1::,-1]))
                ai1.append(np.mean(np.array(plot.loadText(f"{name}/{par}jdd_AI_{i}.son"))[1::,-1][1::3]))
                ai2.append(np.mean(np.array(plot.loadText(f"{name}/{par}jdd_AI_{i}.son"))[1::,-1][2::3]))
    



# Void fraction and interfacial area concentration plots

In [ ]:
# Colonnes : Titre, alpha, d, U (contient U_l et U_g), u'l
  
color_group1 = "darkorchid"
color_group2 = "royalblue"
marker_conf =  { "PolyMAC_ICE" : "--" }

# Experimental data
zexp=[0,1.114,1.815,2.217,2.60,2.98]
alp1exp=[0.,0.0208,0.1108,0.1252,0.1072,0.0928]
alp2exp=[0.,0.,0.,0.006,0.146,0.316]
ai1exp=[0.,48.,201.6,218.1,182.7,154.]
ai2exp=[0.,0.,2.38,2.38,50.,76.2]


plt.figure(figsize = (12,6))

plt.subplot(221)
plt.plot(zexp, alp1exp, "^", color = color_group1,  label = "Experimental data")
plt.plot(zexp, al1, "o", label = f"Average in section SETS", color = color_group1)

for s in dt_ev.keys():
        f = os.open(f"{build_dir}/{s}/{par}jdd.perf", os.O_RDONLY)
        time_loc = str(float(os.read(f, 200).split()[3])/60)[:5]
        plt.plot(np.array(sim[s]["z"]), sim[s]['alp1'], marker_conf[s], linewidth = 3, label = f"Center line {s.replace('_',' ')}", color = color_group1)

plt.legend(fontsize = 8)
plt.xlabel("Height (m)")
plt.ylabel("Void fraction")
plt.title("Comparison of group 1 simulations with experiment")
    
plt.subplot(222)
plt.plot(zexp, alp2exp, "^", color = color_group2,  label = "Experimental data")
plt.plot(zexp, al2, "o", label = f"Average in section ICE", color = color_group2)

for s in dt_ev.keys():
        plt.plot(np.array(sim[s]["z"]), sim[s]['alp2'], marker_conf[s], linewidth = 3, label = f"Center line {s.replace('_',' ')}", color = color_group2)

plt.legend(fontsize = 8)
plt.xlabel("Height (m)")
plt.ylabel("Void fraction")
plt.title("Comparison of group 2 simulations with experiment")

plt.subplot(223)
plt.plot(zexp, ai1exp, "^", color = color_group1,  label = "Experimental data")
plt.plot(zexp, ai1, "o", label = f"Average in section ICE", color = color_group1)

for s in dt_ev.keys():
        f = os.open(f"{build_dir}/{s}/{par}jdd.perf", os.O_RDONLY)
        time_loc = str(float(os.read(f, 200).split()[3])/60)[:5]
        plt.plot(np.array(sim[s]["z"]), sim[s]['ai1'], marker_conf[s], linewidth = 3, label = f"Center line {s.replace('_',' ')} ", color = color_group1)

plt.legend(fontsize = 8)
plt.xlabel("Height (m)")
plt.ylabel("Interfacial area concentration ($m^{-1}$)")

plt.subplot(224)
plt.plot(zexp, ai2exp, "^", color = color_group2,  label = "Experimental data")
plt.plot(zexp, ai2, "o", label = f"Average in section ICE", color = color_group2)

for s in dt_ev.keys():
        plt.plot(np.array(sim[s]["z"]), sim[s]['ai2'], marker_conf[s], linewidth = 3, label = f"Center line {s.replace('_',' ')}", color = color_group2)

plt.legend(fontsize = 8)
plt.xlabel("Height (m)")
plt.ylabel("Interfacial area concentration ($m^{-1}$)")


plt.tight_layout()

# Diameters plots

In [ ]:
# Colonnes : Titre, alpha, d, U (contient U_l et U_g), u'l

    
color_group1 = "darkorchid"
color_group2 = "royalblue"
marker_conf =  { "VDF_ICE" : "--", "VDF_SETS" : "-", "PolyMAC_ICE" : "--" , "PolyMAC_SETS" : "-"}

plt.figure(figsize = (12,6))
db1exp=[]
db2exp=[]
db1=[]
db2=[]
for i in range(len(zexp)):
    if (ai1exp[i] != 0.) :
        db1exp.append(6.*alp1exp[i]/ai1exp[i]*1000)
    else : 
        db1exp.append(0.)
        
    if (ai1[i] != 0.) :
        db1.append(6.*al1[i]/ai1[i]*1000)
    else : 
        db1.append(0.)   
        
    if (ai2exp[i] != 0.) :
        db2exp.append(6.*alp2exp[i]/ai2exp[i]*1000)
    else : 
        db2exp.append(0.)   
        
    if (ai2[i] != 0.) :
        db2.append(6.*al2[i]/ai2[i]*1000)
    else : 
        db2.append(0.) 
       
    
plt.subplot(121)
plt.plot(zexp, db1exp, "^", color = color_group1,  label = "Experimental data")
plt.plot(zexp, db1, "o", label = f"Average in section ICE", color = color_group1)

for s in dt_ev.keys():
        plt.plot(np.array(sim[s]["z"]), sim[s]['db1']*1000, marker_conf[s], linewidth = 3, label = f"Center line {s.replace('_',' ')}", color = color_group1)

plt.legend(fontsize = 8)
plt.xlabel("Height (m)")
plt.ylabel("Bubble diameter group 1 (mm)")
plt.title("Group 1 bubble diameter")
    
plt.subplot(122)
plt.plot(zexp, db2exp, "^", color = color_group2,  label = "Experimental data")
plt.plot(zexp, db2, "o", label = f"Average in section ICE", color = color_group2)

for s in dt_ev.keys():
        plt.plot(np.array(sim[s]["z"]), sim[s]['db2']*1000, marker_conf[s], linewidth = 3, label = f"Center line {s.replace('_',' ')}", color = color_group2)

plt.legend(fontsize = 8)
plt.xlabel("Height (m)")
plt.ylabel("Bubble diameter group 2 (mm)")
plt.title("Group 2 bubble diameter ")

plt.tight_layout()

# Temperature profiles

In [ ]:
# Colonnes : Titre, alpha, d, U (contient U_l et U_g), u'l

    
color_group1 = "darkorchid"
color_group2 = "royalblue"
marker_conf =  { "PolyMAC_ICE" : "--" }

plt.figure(figsize = (12,6))
     

plt.subplot(121)

for s in dt_ev.keys():
        plt.plot(np.array(sim[s]["z"]), sim[s]['Tg1'], marker_conf[s], linewidth = 3, label = f"Center line {s.replace('_',' ')}", color = color_group1)
        plt.plot(np.array(sim[s]["z"]), sim[s]['Tl'], marker_conf[s], linewidth = 3, label = f"Liquid, Center line {s.replace('_',' ')}", color = 'black')

plt.legend(fontsize = 8)
plt.xlabel("Height (m)")
plt.ylabel("Temperature (°C)")
plt.title("Group 1")
    
plt.subplot(122)

for s in dt_ev.keys():
        plt.plot(np.array(sim[s]["z"]), sim[s]['Tg2'], marker_conf[s], linewidth = 3, label = f"Center line {s.replace('_',' ')}", color = color_group2)
        plt.plot(np.array(sim[s]["z"]), sim[s]['Tl'], marker_conf[s], linewidth = 3, label = f"Liquid, Center line {s.replace('_',' ')}", color = 'black')

plt.legend(fontsize = 8)
plt.xlabel("Height (m)")
plt.ylabel("Temperature (°C)")
plt.title("Group 2")

plt.tight_layout()